# 01. 데이터 수집 - pykrx를 활용한 한국 주식 데이터 수집

## 학습 목표
- pykrx를 사용하여 다양한 종류의 주식 데이터를 수집하는 방법 학습
- 주가, 재무, 투자자별 매매 데이터 수집 및 저장
- 대량 데이터 수집 시 효율적인 처리 방법

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pykrx import stock
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# 날짜 설정
end_date = datetime.now()
start_date = end_date - timedelta(days=365*2)  # 2년 데이터

start_str = start_date.strftime('%Y%m%d')
end_str = end_date.strftime('%Y%m%d')

print(f"데이터 수집 기간: {start_str} ~ {end_str}")

## 1. 종목 리스트 수집

In [ ]:
def get_all_stocks():
    """
    KOSPI와 KOSDAQ 모든 종목 리스트 가져오기
    """
    stocks = []
    
    for market in ['KOSPI', 'KOSDAQ']:
        tickers = stock.get_market_ticker_list(market=market)
        print(f"{market} 종목 수: {len(tickers)}")
        
        for ticker in tqdm(tickers[:20], desc=f"{market} 종목 정보"):  # 데모용 20개
            try:
                name = stock.get_market_ticker_name(ticker)
                
                # 시가총액과 기본 정보
                cap_df = stock.get_market_cap_by_ticker(end_str, market)
                if ticker in cap_df.index:
                    cap_info = cap_df.loc[ticker]
                    
                    stocks.append({
                        'ticker': ticker,
                        'name': name,
                        'market': market,
                        'market_cap': cap_info['시가총액'],
                        'shares': cap_info['상장주식수']
                    })
                    
                time.sleep(0.1)
            except Exception as e:
                print(f"Error with {ticker}: {e}")
                continue
    
    return pd.DataFrame(stocks)

# 종목 리스트 수집
stock_list = get_all_stocks()
print(f"\n총 수집 종목: {len(stock_list)}")
print(stock_list.head(10))

## 2. 주가 데이터 수집

In [ ]:
def get_price_data(ticker, start, end):
    """
    종목의 주가 데이터 수집
    
    Returns:
        DataFrame: 시가, 고가, 저가, 종가, 거래량, 등락률, 거래대금
    """
    try:
        df = stock.get_market_ohlcv_by_date(start, end, ticker)
        df['ticker'] = ticker
        
        # 추가 지표 계산
        df['MA20'] = df['종가'].rolling(window=20).mean()
        df['MA60'] = df['종가'].rolling(window=60).mean()
        df['MA120'] = df['종가'].rolling(window=120).mean()
        
        # 52주 최고/최저
        df['52주최고'] = df['고가'].rolling(window=252, min_periods=1).max()
        df['52주최저'] = df['저가'].rolling(window=252, min_periods=1).min()
        df['52주최고대비'] = df['종가'] / df['52주최고']
        
        # 거래량 관련
        df['거래량MA20'] = df['거래량'].rolling(window=20).mean()
        df['거래량비율'] = df['거래량'] / df['거래량MA20']
        
        return df
    except Exception as e:
        print(f"Error getting price data for {ticker}: {e}")
        return pd.DataFrame()

# 삼성전자 예시
samsung_price = get_price_data('005930', start_str, end_str)
print("삼성전자 주가 데이터:")
print(samsung_price.tail())
print(f"\n52주 최고가 대비: {samsung_price['52주최고대비'].iloc[-1]:.2%}")

## 3. 재무 데이터 수집

In [ ]:
def get_fundamental_data(ticker, start, end):
    """
    종목의 기본적 재무 지표 수집
    
    Returns:
        DataFrame: PER, PBR, EPS, BPS, DIV, DPS
    """
    try:
        df = stock.get_market_fundamental_by_date(start, end, ticker)
        df['ticker'] = ticker
        
        # EPS 성장률 계산
        df['EPS_YoY'] = df['EPS'].pct_change(periods=252)  # 연간 성장률
        df['EPS_QoQ'] = df['EPS'].pct_change(periods=63)   # 분기 성장률
        
        # ROE 계산 (EPS / BPS)
        df['ROE'] = df['EPS'] / df['BPS'].shift(252)  # 전년도 BPS 대비
        
        return df
    except Exception as e:
        print(f"Error getting fundamental data for {ticker}: {e}")
        return pd.DataFrame()

# 삼성전자 재무 데이터
samsung_fund = get_fundamental_data('005930', start_str, end_str)
print("삼성전자 재무 지표:")
print(samsung_fund.tail())
print(f"\n최근 EPS 연간 성장률: {samsung_fund['EPS_YoY'].iloc[-1]:.2%}")
print(f"최근 ROE: {samsung_fund['ROE'].iloc[-1]:.2%}")

## 4. 투자자별 매매 동향

In [ ]:
def get_investor_trading(ticker, start, end):
    """
    투자자별 매매 동향 수집
    
    Returns:
        DataFrame: 기관, 외국인, 개인 등 투자자별 매매 데이터
    """
    try:
        # 투자자별 매매 거래량
        df = stock.get_market_trading_value_by_date(start, end, ticker, detail=True)
        df['ticker'] = ticker
        
        # 순매수 계산
        df['기관_순매수'] = df['기관'] 
        df['외국인_순매수'] = df['외국인계'] if '외국인계' in df.columns else df.get('외국인', 0)
        df['개인_순매수'] = df['개인']
        
        # 누적 순매수
        df['기관_누적'] = df['기관_순매수'].cumsum()
        df['외국인_누적'] = df['외국인_순매수'].cumsum()
        df['개인_누적'] = df['개인_순매수'].cumsum()
        
        # 최근 20일 순매수 합계
        df['기관_20일'] = df['기관_순매수'].rolling(window=20).sum()
        df['외국인_20일'] = df['외국인_순매수'].rolling(window=20).sum()
        
        return df
    except Exception as e:
        print(f"Error getting investor data for {ticker}: {e}")
        return pd.DataFrame()

# 삼성전자 투자자별 매매
samsung_investor = get_investor_trading('005930', start_str, end_str)
print("삼성전자 투자자별 매매 동향:")
print(samsung_investor.tail())
print(f"\n최근 20일 기관 순매수: {samsung_investor['기관_20일'].iloc[-1]:,.0f}")
print(f"최근 20일 외국인 순매수: {samsung_investor['외국인_20일'].iloc[-1]:,.0f}")

## 5. 업종 데이터 수집

In [ ]:
def get_sector_performance():
    """
    업종별 성과 데이터 수집
    """
    # 업종 리스트
    sectors = stock.get_index_ticker_list(market='KOSPI')
    
    sector_data = []
    for sector_code in tqdm(sectors[:10], desc="업종 데이터 수집"):  # 데모용 10개
        try:
            name = stock.get_index_ticker_name(sector_code)
            
            # 업종 지수 데이터
            sector_ohlcv = stock.get_index_ohlcv_by_date(start_str, end_str, sector_code)
            
            if not sector_ohlcv.empty:
                # 성과 계산
                returns_1m = (sector_ohlcv['종가'].iloc[-1] / sector_ohlcv['종가'].iloc[-20] - 1) * 100
                returns_3m = (sector_ohlcv['종가'].iloc[-1] / sector_ohlcv['종가'].iloc[-60] - 1) * 100 if len(sector_ohlcv) > 60 else 0
                returns_6m = (sector_ohlcv['종가'].iloc[-1] / sector_ohlcv['종가'].iloc[-120] - 1) * 100 if len(sector_ohlcv) > 120 else 0
                
                sector_data.append({
                    'sector_code': sector_code,
                    'sector_name': name,
                    'returns_1m': returns_1m,
                    'returns_3m': returns_3m,
                    'returns_6m': returns_6m,
                    'last_close': sector_ohlcv['종가'].iloc[-1]
                })
                
            time.sleep(0.1)
        except Exception as e:
            print(f"Error with sector {sector_code}: {e}")
            continue
    
    return pd.DataFrame(sector_data)

# 업종 성과 데이터
sector_df = get_sector_performance()
print("\n업종별 성과:")
print(sector_df.sort_values('returns_6m', ascending=False))

## 6. 통합 데이터 수집 함수

In [ ]:
class StockDataCollector:
    """
    종목 데이터를 통합적으로 수집하는 클래스
    """
    
    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.name = stock.get_market_ticker_name(ticker)
        
    def collect_all_data(self):
        """
        모든 종류의 데이터를 수집
        """
        print(f"\n{self.name}({self.ticker}) 데이터 수집 중...")
        
        data = {}
        
        # 1. 주가 데이터
        print("  - 주가 데이터 수집...")
        data['price'] = get_price_data(self.ticker, self.start_date, self.end_date)
        
        # 2. 재무 데이터
        print("  - 재무 데이터 수집...")
        data['fundamental'] = get_fundamental_data(self.ticker, self.start_date, self.end_date)
        
        # 3. 투자자별 매매
        print("  - 투자자별 매매 데이터 수집...")
        data['investor'] = get_investor_trading(self.ticker, self.start_date, self.end_date)
        
        return data
    
    def get_latest_metrics(self, data):
        """
        최신 지표 요약
        """
        metrics = {}
        
        if not data['price'].empty:
            metrics['현재가'] = data['price']['종가'].iloc[-1]
            metrics['52주최고대비'] = data['price']['52주최고대비'].iloc[-1]
            metrics['거래량비율'] = data['price']['거래량비율'].iloc[-1]
            metrics['20일평균대비'] = data['price']['종가'].iloc[-1] / data['price']['MA20'].iloc[-1]
        
        if not data['fundamental'].empty:
            metrics['PER'] = data['fundamental']['PER'].iloc[-1]
            metrics['PBR'] = data['fundamental']['PBR'].iloc[-1]
            metrics['EPS_YoY'] = data['fundamental']['EPS_YoY'].iloc[-1]
            metrics['ROE'] = data['fundamental']['ROE'].iloc[-1]
        
        if not data['investor'].empty:
            metrics['기관20일'] = data['investor']['기관_20일'].iloc[-1]
            metrics['외국인20일'] = data['investor']['외국인_20일'].iloc[-1]
        
        return metrics

# 사용 예시
collector = StockDataCollector('005930', start_str, end_str)
samsung_data = collector.collect_all_data()
samsung_metrics = collector.get_latest_metrics(samsung_data)

print("\n삼성전자 주요 지표:")
for key, value in samsung_metrics.items():
    if isinstance(value, float):
        if key.endswith('율') or key.endswith('YoY') or key == 'ROE' or '대비' in key:
            print(f"  {key}: {value:.2%}")
        else:
            print(f"  {key}: {value:,.2f}")
    else:
        print(f"  {key}: {value}")

## 7. 대량 데이터 수집 및 저장

In [ ]:
def collect_multiple_stocks(ticker_list, start_date, end_date):
    """
    여러 종목의 데이터를 수집하고 저장
    """
    all_data = {}
    failed_tickers = []
    
    for ticker in tqdm(ticker_list, desc="종목 데이터 수집"):
        try:
            collector = StockDataCollector(ticker, start_date, end_date)
            data = collector.collect_all_data()
            metrics = collector.get_latest_metrics(data)
            
            all_data[ticker] = {
                'name': collector.name,
                'data': data,
                'metrics': metrics
            }
            
            time.sleep(0.5)  # API 호출 제한 방지
            
        except Exception as e:
            print(f"\nError with {ticker}: {e}")
            failed_tickers.append(ticker)
            continue
    
    print(f"\n수집 완료: {len(all_data)}/{len(ticker_list)} 종목")
    if failed_tickers:
        print(f"실패한 종목: {failed_tickers}")
    
    return all_data

# 테스트: 상위 5개 종목
test_tickers = stock_list['ticker'].head(5).tolist()
collected_data = collect_multiple_stocks(test_tickers, start_str, end_str)

# 결과 요약
summary_df = pd.DataFrame([
    {
        'ticker': ticker,
        'name': data['name'],
        **data['metrics']
    }
    for ticker, data in collected_data.items()
])

print("\n수집된 데이터 요약:")
print(summary_df)

## 8. 데이터 저장 및 로드

In [ ]:
def save_data(data, filename_prefix):
    """
    수집한 데이터를 파일로 저장
    """
    import pickle
    import os
    
    # 데이터 디렉토리 생성
    os.makedirs('data', exist_ok=True)
    
    # Pickle로 저장 (전체 데이터)
    with open(f'data/{filename_prefix}_all.pkl', 'wb') as f:
        pickle.dump(data, f)
    
    # CSV로 요약 저장
    summary_list = []
    for ticker, stock_data in data.items():
        if 'metrics' in stock_data:
            summary_list.append({
                'ticker': ticker,
                'name': stock_data['name'],
                **stock_data['metrics']
            })
    
    if summary_list:
        summary_df = pd.DataFrame(summary_list)
        summary_df.to_csv(f'data/{filename_prefix}_summary.csv', index=False, encoding='utf-8-sig')
    
    print(f"데이터 저장 완료: data/{filename_prefix}_*")

def load_data(filename_prefix):
    """
    저장된 데이터 로드
    """
    import pickle
    
    with open(f'data/{filename_prefix}_all.pkl', 'rb') as f:
        data = pickle.load(f)
    
    print(f"데이터 로드 완료: {len(data)} 종목")
    return data

# 데이터 저장
save_data(collected_data, 'canslim_demo')

# 데이터 로드 테스트
loaded_data = load_data('canslim_demo')
print(f"로드된 종목: {list(loaded_data.keys())}")

## 마무리

### 학습 내용 정리
1. **pykrx 주요 함수들**
   - `get_market_ticker_list()`: 종목 리스트
   - `get_market_ohlcv_by_date()`: 주가 데이터
   - `get_market_fundamental_by_date()`: 재무 지표
   - `get_market_trading_value_by_date()`: 투자자별 매매

2. **데이터 수집 시 주의사항**
   - API 호출 제한을 고려한 딜레이 추가
   - 예외 처리로 안정적인 수집
   - 대량 데이터는 저장 후 재사용

3. **계산된 지표들**
   - 이동평균선 (MA20, MA60, MA120)
   - 52주 최고/최저 대비
   - EPS 성장률 (YoY, QoQ)
   - ROE
   - 투자자별 누적 매수

### 다음 단계
- **02_current_earnings_analysis.ipynb**: C 지표 (현재 분기 실적) 분석
- 수집한 데이터를 활용한 CAN SLIM 지표 계산